# Piping a prompt, model, and output parser

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [3]:
# Chat Model definition
chat = ChatOpenAI(
    model = 'gpt-4',
    temperature=0,
    seed=25,
    max_completion_tokens=100
)

# Comma Seperated Parser instructions
instructions = CommaSeparatedListOutputParser().get_format_instructions()

# Defining chat prompt
chat_prompt = ChatPromptTemplate.from_messages([('human', 'I want the list of 10 tourist places in {place}.\n' + instructions)])

# Defining the Output Parser object
output_parser = CommaSeparatedListOutputParser()

# Creaing a chain
chain = chat_prompt | chat | output_parser

# Sending the prompt and get the response
response = chain.invoke({'place': 'Delhi'})
print(response)

['Red Fort', 'Qutub Minar', 'India Gate', 'Lotus Temple', "Humayun's Tomb", 'Jama Masjid', 'Akshardham Temple', 'Rashtrapati Bhavan', 'Jantar Mantar', 'Raj Ghat']


# Batching

In [4]:
# Chat Model definition
chat = ChatOpenAI(
    model = 'gpt-4',
    temperature=0,
    seed=25
)

# Defining chat prompt
chat_prompt = ChatPromptTemplate.from_messages([('human', 'I want the list of 10 tourist places in {place} during {season}.\n')])

# Defining the chain
chain = chat_prompt | chat

# Generating the response
response = chain.batch([{'place': 'Kolkata', 'season': 'Christmas'}, {'place': 'Kashmir', 'season': 'Winters'}])
for aiMessage in response:
    print(aiMessage.content)
    print('\n'*2)

1. St. Paul's Cathedral: This iconic cathedral is beautifully decorated during Christmas and hosts midnight mass on Christmas Eve.

2. Park Street: Known as the food street of Kolkata, Park Street is beautifully lit up during Christmas and is a hub of festivities.

3. New Market: This is a great place to shop for Christmas decorations and gifts. The market is beautifully decorated and has a festive atmosphere.

4. Indian Museum: One of the oldest museums in the world, it is a great place to learn about India's rich history and culture.

5. Victoria Memorial: This grand marble building is a must-visit. It is beautifully lit up during Christmas.

6. Nicco Park: This amusement park is a great place to enjoy with family during Christmas. They often have special events and shows during the festive season.

7. Howrah Bridge: The iconic bridge is a must-visit. It is beautifully lit up during Christmas and offers a great view of the city.

8. Eden Gardens: This famous cricket stadium often hos

In [5]:
# Chat Model definition
chat = ChatOpenAI(
    model = 'gpt-4',
    temperature=0,
    seed=25,
    max_completion_tokens=200
)

# Defining chat prompt
chat_prompt = ChatPromptTemplate.from_messages([('human', 'Give me the history of {location} in 50 words\n')])

# Defining the chain
chain = chat_prompt | chat

# Generating the response
response = chain.stream({'location':'kolkata'})

for i in response:
    print(i.content, end='')

Kolkata, formerly known as Calcutta, was established in 1690 by the British East India Company. It served as the capital of British India until 1911. Known for its literary, artistic, and revolutionary heritage, Kolkata is the cultural capital of India. It witnessed significant political and social movements during the British rule and post-independence era.

# Runnable and RunnableSequence

In [6]:
type(chain)

langchain_core.runnables.base.RunnableSequence

In [7]:
type(chat)

langchain_openai.chat_models.base.ChatOpenAI

In [8]:
type(chat_prompt)

langchain_core.prompts.chat.ChatPromptTemplate

# Piping chains and RunnablePassThrough Class

In [9]:
# Chat Model definition
chat = ChatOpenAI(
    model = 'gpt-4',
    temperature=0,
    seed=25,
    max_completion_tokens=200
)

# Chat prompt templates
chat_tools = ChatPromptTemplate.from_template('''
What are the best five inportant tools a {job title} needs?
Answer only by listing the tools
''')

chat_strategy = ChatPromptTemplate.from_template('''
Cpnsidering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

# Defining output parser
string_parser = StrOutputParser()

# Chain creations
chain_tools = chat_tools | chat | string_parser | {'tools': RunnablePassthrough()}
chain_strategy = chat_strategy | chat | string_parser

# Combined Chain
chain_combined = chain_tools | chain_strategy

# Generating response
print(chain_combined.invoke({'job title': 'data scientist'}))

1. Python: Start by learning the basics of Python, such as variables, data types, loops, and functions. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding problems on platforms like LeetCode or HackerRank. Work on small projects, like a simple game or a personal website, to apply what you've learned. As you get more comfortable, learn more advanced topics like object-oriented programming and data structures.

2. R Programming Language: Start with the basics of R, such as data types, vectors, matrices, lists, and data frames. Use online resources like DataCamp, Coursera, or edX for structured learning. Practice coding problems on platforms like R-exercises or Kaggle. Work on data analysis or data visualization projects to apply what you've learned. As you get more comfortable, learn more advanced topics like statistical modeling and machine learning.

3. SQL: Start by learning the basics of SQL, such as SELECT


# Graphing Runnables using grandalf library

In [10]:
# Loading libraries
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Defining the chat model
chat = ChatOpenAI(
    model = 'gpt-4',
    temperature=0,
    seed=25,
    max_completion_tokens=200
)

# Chat prompt templates
chat_tools = ChatPromptTemplate.from_template('''
What are the best five inportant tools a {job title} needs?
Answer only by listing the tools
''')

chat_strategy = ChatPromptTemplate.from_template('''
Cpnsidering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

# String Output Parser
string_parser = StrOutputParser()

# Defining the chain 
chain = (chat_tools | chat | string_parser | {'tools': RunnablePassthrough()} | chat_strategy | chat | string_parser)

# Graphing chain
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       

# Runnable Prallel

In [11]:
# Loading libraries
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

# Defining the chat model
chat = ChatOpenAI(
    model = 'gpt-4',
    temperature=0,
    seed=25,
    max_completion_tokens= 100
)

# Chat prompt templates
chat_books = ChatPromptTemplate.from_template(
'''
Suggest three of the best intermediate-level {programming language} books. 
Answer only by listing the tools.
''')

chat_project = ChatPromptTemplate.from_template(
'''
Suggest three intermediate {programming language} projects suitable for intermedia-level programmers
Answer only by listing the tools.
''')

# String Output Parser
string_parser = StrOutputParser()

# Defining chains
chain_books = chat_books | chat | string_parser
chain_projects = chat_project | chat | string_parser

chain_combined = RunnableParallel({'books': chain_books, 'projects': chain_projects})

# Runnable graph
chain_combined.get_graph().print_ascii()

# getting response
response = chain_combined.invoke({'programming language': 'Python'})

# Printing the response
print(response['books'])
print('\n')
print(response['projects'])

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

# Piping a RunnableParallel with other Runnable

In [13]:
# Loading libraries
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

# Defining the chat model
chat = ChatOpenAI(
    model = 'gpt-4',
    temperature=0,
    seed=25,
    max_completion_tokens= 500
)

# Chat prompt templates
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

chat_template_time = ChatPromptTemplate.from_template(
     '''
     I'm an intermediate level programmer.
     Consider the following literature:
     {books}
     Also, consider the following projects:
     {projects}
     Roughly how much time would it take me to complete the literature and the projects?
     
     '''
)

# String Output Parser
string_parser = StrOutputParser()

# Defining chains
chain_books = chat_books | chat | string_parser
chain_projects = chat_project | chat | string_parser

chain_combined = ({'books':chain_books, 
                'projects':chain_projects}
              | chat_template_time 
              | chat 
              | string_parser
             )

# Runnable graph
chain_combined.get_graph().print_ascii()

# getting response
response = chain_combined.invoke({'programming language':'Python'})

# Printing the response
print('\n')
print(response)

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            